# get_frequently_mutated Final 

In [2]:
import cptac
en = cptac.Endometrial()
import pandas as pd
import numpy as np

Welcome to cptac, a python package for disseminating cancer
proteogenomics data. To view available datasets, enter
'cptac.list_data()'. Extensive tutorials are available at
https://github.com/PayneLab/cptac/tree/master/doc

******
Version: 0.4.5
******
Checking that data files are up-to-date...
100% [..................................................................................] 649 / 649
Data check complete.
endometrial data version: 2.1

Loading acetylproteomics data...
Loading clinical data...
Loading CNA data...
Loading definitions data...
Loading miRNA data...
Loading phosphoproteomics_gene data...
Loading phosphoproteomics_site data...
Loading proteomics data...
Loading somatic data...
Loading somatic_binary data...
Loading transcriptomics_circular data...
Loading transcriptomics_linear data...

 ******PLEASE READ******
CPTAC is a community resource project and data are made available
rapidly after generation for community research use. The embargo
allows exploring and utiliz

In [12]:
def get_frequently_mutated(cancer_type, cutoff=.1):  
    """
    take cancer object and find the frequently 
    mutated genes in total tumors compared to the cutoff.
        
        Parameters:
        cancer_type (object): object of cancer class from cptac module 
        cutoff (float): used as comparison to determine status of gene mutation frequency
        
        Returns:
        freq_mutated_df (pd.DataFrame): DataFrame of frequently mutated genes passing the cutoff
            and Total_Mutated (mutated genes / total tumors), percent Truncated, and percent Missence
        
        There are many types of mutations catagorized into the columns Truncated and Missence. 
        The Truncated column includes: Frame_Shift_Del, Frame_Shift_Ins, Splice_Site, Nonsense_Mutation, Nonstop_Mutation
        The Missence column includes: In_Frame_Del, In_Frame_Ins, Missense_Mutation
        These columns count multiple mutations of one gene in the same sample, so percentages in the last two columns may 
        exceed the Total_Mutated column(which only counts if the gene was mutated once)"""    
    
    # get data frames
    somatic_mutations = cancer_type.get_mutations()
    sample_status_map = cancer_type._get_sample_status_map()
    merged_mutations = somatic_mutations.join(sample_status_map, how="left") 
    """
    # standardize mutation names 
    if cancer_type.get_tumor_type() == 'Colon':
        mutation_equivalents = {'frameshift substitution': 'Frame_Shift_Del' , 'frameshift deletion': 'Frame_Shift_Del', 
            'frameshift insertion': 'Frame_Shift_Ins', 'stopgain': 'Nonsense_Mutation ', 'stoploss':'Nonstop_Mutation',
            'nonsynonymous SNV': 'Missense_Mutation','nonframeshift insertion': 'In_Frame_Ins',
            'nonframeshift deletion': 'In_Frame_Del', 'nonframeshift substitution': 'Missense_Mutation'}
        merged_mutations = merged_mutations.replace(to_replace = mutation_equivalents)
    """  
    # get list of unique genes
    unique_genes = somatic_mutations['Gene'].unique()
    
    # get total tumors/patients
    sample_status_series = sample_status_map.value_counts()
    total_tumor_patients = sample_status_series[0]
        
    # find frequently mutated genes and their total mutated fraction. Create lists for frequently mutated genes and fraction.
    freq_mut = [] # .where, pandas already vectorized, faster to use their function than a for loop
    total_fraction_mutated = []
    for gene in unique_genes:
        gene_mutated = merged_mutations.loc[merged_mutations['Gene'] == gene]
        gene_mutated = gene_mutated.index.unique()
        num_gene_mutated = len(gene_mutated)
        fraction = (num_gene_mutated / total_tumor_patients)
        if fraction > cutoff:
            freq_mut.append(gene)
            total_fraction_mutated.append(fraction)
    
    # find truncated fraction
    list_missence_mut = ['In_Frame_Del', 'In_Frame_Ins', 'Mutation']
    
    truncated = []
    missence = []
    for gene in freq_mut:
        gene_mut = merged_mutations.loc[merged_mutations['Gene'] == gene] #possibly change var name
        
        truncated_df = gene_mut.loc[(gene_mut['Mutation'] not in list_missence_mut)]
        samples_trunc = truncated_df.index.unique()
        num_trunc_mut = len(samples_trunc)
        fraction_trunc = (num_trunc_mut / total_tumor_patients)
        truncated.append(fraction_trunc)
        
        missence_mutations = gene_mutated.loc[(gene_mutated['Mutation'] in list_missence_mut)]
        samples_miss = missence_mutations.index.unique()
        num_miss_mut = len(samples_miss)
        fraction_miss = (num_miss_mut / total_tumor_patients)
        missence.append(fraction_miss)
        
        
    # create dataframe
    merged_lists = list(zip(freq_mut, total_fraction_mutated, truncated, missence))
    freq_mutated_df = pd.DataFrame(merged_lists, columns =['Gene', 'Fraction_Mutated', 'Truncation', 'Missence'])
    freq_mutated_df.name = 'frequently_mutated'
                   
    return freq_mutated_df

Test Endometrial

In [49]:
somatic_mutations = en.get_mutations()
somatic_mutations.head()
unique_genes = somatic_mutations['Gene'].unique()
#somatic_mutations.index.drop_duplicates()
list1 = ['A1CF', 'KRAS']
#n = list(map(lambda gene: (somatic_mutations.loc[somatic_mutations['Gene'] == gene]).index.unique(), unique_genes))
sorted_mut = somatic_mutations.sort_values(by='Gene')
print(sorted_mut.head())
len(somatic_mutations.loc[somatic_mutations['Gene'] == 'A1CF'].index.unique())
n = list(map(lambda gene: somatic_mutations.loc[somatic_mutations['Gene'] == gene].index.unique(), list1))
b = sorted_mut.groupby('Sample_ID').nunique()
b.drop('Location', axis = 1)

           Gene           Mutation  Location
Sample_ID                                   
S055       A1BG  Missense_Mutation   p.E298K
S059       A1CF  Missense_Mutation   p.F487L
S059       A1CF  Missense_Mutation   p.S236Y
S070        A2M  Missense_Mutation  p.G1247S
S059        A2M  Missense_Mutation   p.K209N


,Gene,Mutation
Sample_ID,,
S001,754,6
S002,340,7
S003,30,6
S005,40,7
S006,51,5
S007,32,6
S008,40,6
S009,55,6
S010,483,7


In [119]:
somatic_mutations = en.get_mutations()
#move samples to col so can be counted
df = somatic_mutations.reset_index()

 # group by gene and count unique samples
genes_df = df.groupby(['Gene']).nunique()

#format
genes_df = genes_df.rename(columns={"Sample_ID": "Unique_Samples_Mut"})
genes_df = genes_df.drop(['Location','Gene', 'Mutation'], axis = 1)

#test
gene = 'ARID1A'
c = genes_df.loc[gene:gene,:]

genes_df = genes_df.apply(lambda x: x / 95)




In [140]:
somatic_mutations = en.get_mutations()
#move samples to col so can be counted
df = somatic_mutations.reset_index()

 # group by gene and count unique samples
genes_df = df.groupby(['Gene']).nunique()

#format
genes_df = genes_df.rename(columns={"Sample_ID": "Unique_Samples_Mut"})
genes_df = genes_df.drop(['Location','Gene', 'Mutation'], axis = 1)

#test
gene = 'ARID1A'
c = genes_df.loc[gene:gene,:]

genes_df = genes_df.apply(lambda x: x / 95)
filtered = genes_df.where(lambda x: x > .15)
filtered = filtered.dropna()
filtered

,Unique_Samples_Mut
Gene,
AHNAK,0.157895
ARID1A,0.452632
CCDC168,0.168421
CTCF,0.284211
CTNNB1,0.305263
DNAH17,0.157895
DOCK3,0.200000
FBXW7,0.189474
HERC2,0.178947


In [138]:
somatic_mutations = en.get_mutations()
#move samples to col so can be counted
df = somatic_mutations.reset_index()

# 2 VALUES FOR MUT
missence_equivalents = 

 # group by gene and count unique samples
genes_df = df.groupby(['Gene', 'Sample_ID']).nunique()


.replace(to_replace = mutation_equivalents)
  


Sample_ID  Gene  Mutation  Location
Gene    Sample_ID                                     
A1BG    S055               1     1         1         1
A1CF    S059               1     1         1         2
A2M     S023               1     1         1         1
        S059               1     1         2         3
        S066               1     1         1         1
        S070               1     1         1         1
        S074               1     1         1         1
A2ML1   S022               1     1         1         1
        S023               1     1         1         2
        S055               1     1         1         2
        S059               1     1         1         1
        S066               1     1         1         1
A3GALT2 S022               1     1         1         1
        S059               1     1         1         1
A4GALT  S073               1     1         1         1
A4GNT   S023               1     1         1         1
        S057               1     1         1         1
        S099               1     1         1         1
AAAS    S022               1     1         1         1
        S059               1     1         1         1
AACS    S022               1     1         1         2
        S059               1     1         1         1
AADAC   S055               1     1         1         1
AADACL2 S012               1     1         1         1
        S023               1     1         1         1
        S059               1     1         1         2
        S066               1     1         1         1
AADACL3 S022               1     1         1         1
        S059               1     1         1         1
AADACL4 S010               1     1         1         1
...                      ...   ...       ...       ...
ZWILCH  S022               1     1         1         1
        S027               1     1         1         1
        S059               1     1         2         2
ZXDA    S022               1     1         1         1
        S028               1     1         1         1
        S050               1     1         1         1
        S059               1     1         1         1
        S066               1     1         1         1
        S085               1     1         1         1
ZXDB    S010               1     1         1         1
        S012               1     1         1         1
        S022               1     1         1         1
        S037               1     1         1         1
        S055               1     1         1         1
        S066               1     1         1         1
        S088               1     1         1         1
ZXDC    S023               1     1         1         1
        S030               1     1         1         1
        S051               1     1         1         1
        S059               1     1         1         1
        S068               1     1         1         1
ZYG11A  S022               1     1         2         4
        S023               1     1         1         1
ZYG11B  S022               1     1         1         2
        S099               1     1         1         1
ZYX     S081               1     1         1         1
ZZEF1   S022               1     1         1         2
        S030               1     1         1         1
        S094               1     1         1         1
ZZZ3    S055               1     1         1         1

[42185 rows x 4 columns]

In [87]:
g = somatic_mutations.groupby('Gene').groups
g
g['PTEN'] = len(g['PTEN'].value_counts())
#.size()

{'A1BG': Index(['S055'], dtype='object', name='Sample_ID'),
 'A1CF': Index(['S059', 'S059'], dtype='object', name='Sample_ID'),
 'A2M': Index(['S023', 'S059', 'S059', 'S059', 'S066', 'S070', 'S074'], dtype='object', name='Sample_ID'),
 'A2ML1': Index(['S022', 'S023', 'S023', 'S055', 'S055', 'S059', 'S066'], dtype='object', name='Sample_ID'),
 'A3GALT2': Index(['S022', 'S059'], dtype='object', name='Sample_ID'),
 'A4GALT': Index(['S073'], dtype='object', name='Sample_ID'),
 'A4GNT': Index(['S023', 'S057', 'S099'], dtype='object', name='Sample_ID'),
 'AAAS': Index(['S022', 'S059'], dtype='object', name='Sample_ID'),
 'AACS': Index(['S022', 'S022', 'S059'], dtype='object', name='Sample_ID'),
 'AADAC': Index(['S055'], dtype='object', name='Sample_ID'),
 'AADACL2': Index(['S012', 'S023', 'S059', 'S059', 'S066'], dtype='object', name='Sample_ID'),
 'AADACL3': Index(['S022', 'S059'], dtype='object', name='Sample_ID'),
 'AADACL4': Index(['S010', 'S030', 'S055'], dtype='object', name='Sample_ID

In [46]:

df = pd.DataFrame([('bird', 'Falconiformes', 389.0),
                       ('bird', 'Psittaciformes', 24.0),
                       ('mammal', 'Carnivora', 80.2),
                       ('mammal', 'Primates', np.nan),
                       ('mammal', 'Carnivora', 58)],
                      index=['falcon', 'parrot', 'lion', 'monkey', 'leopard'],
                      columns=('class', 'order', 'max_speed'))
df

,class,order,max_speed
falcon,bird,Falconiformes,389.0
parrot,bird,Psittaciformes,24.0
lion,mammal,Carnivora,80.2
monkey,mammal,Primates,NaN
leopard,mammal,Carnivora,58.0


In [55]:
d = df.groupby(['class'])
d.head(2)

,class,order,max_speed
falcon,bird,Falconiformes,389.0
parrot,bird,Psittaciformes,24.0
lion,mammal,Carnivora,80.2
monkey,mammal,Primates,NaN


In [15]:
endo_freq_mutated_df = get_frequently_mutated(en, .25)

NameError: name 'my_list' is not defined

In [ ]:
endo_freq_mutated_df


Test Colon

In [ ]:
colon_freq_mutated_df = get_frequently_mutated("colon")

You have loaded the cptac colon dataset. To view available dataframes,
use cptac.colon.list_data(). To view available functions for accessing
and manipulating the dataframes, use cptac.colon.list_api().
colon data version: Most recent release

Loading cptac colon data:
Loading clinical data...
Loading miRNA data...
Loading mutation data...
Loading mutation_binary data...
Loading phosphoproteomics_normal data...
Loading phosphoproteomics_tumor data...
Loading proteomics_normal data...
Loading proteomics_tumor data...
Loading transcriptomics data...


In [ ]:
colon_freq_mutated_df

Test Ovarian

In [4]:
ovarian_freq_mutated_df = get_frequently_mutated("ovarian")

Welcome to the cptac data service package. Available datasets may be
viewed using cptac.list_data(). In order to access a specific data
set, import a cptac subfolder using either 'import cptac.dataset' or
'from cptac import dataset'.
******
Version: 0.4.1
******
You have loaded the cptac ovarian dataset. To view available
dataframes, use cptac.ovarian.list_data(). To view available functions
for accessing and manipulating the dataframes, use
cptac.ovarian.list_api().
ovarian data version: Most recent release

Loading cptac ovarian data:
Loading clinical data...
Loading cnv data...
Loading phosphoproteomics data...
Loading proteomics data...
Loading somatic_38 data...
Loading transcriptomics data...
Loading treatment data...

 ******PLEASE READ******
CPTAC is a community resource project and data are made available
rapidly after generation for community research use. The embargo
allows exploring and utilizing the data, but the data may not be in a
publication until June 1, 2019. Please 

In [5]:
ovarian_freq_mutated_df

,Gene,Fraction_Mutated,Truncation,Missence
0,TTN,0.279279,0.135135,0.198198
1,TP53,0.693694,0.243243,0.450450
2,MUC16,0.144144,0.054054,0.108108
3,MT-CO1,0.117117,0.027027,0.090090
4,MT-ND5,0.144144,0.081081,0.063063
5,FCGBP,0.108108,0.054054,0.063063
6,MUC5B,0.108108,0.027027,0.081081
7,MUC4,0.306306,0.135135,0.234234
8,MUC17,0.126126,0.036036,0.108108
9,NCOR2,0.108108,0.081081,0.027027
